In [1]:
include("../../src/JWAS.jl")

JWAS

In [2]:
using DataFrames
using JWAS.Datasets
using JWAS

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

In [4]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [29]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [30]:
model_equations = "BW = intercept + age + sex + Animal
                   CW = intercept + age + sex + Animal";

In [31]:
#model    = build_model(model_equations,R[1,1]);
model    = build_model(model_equations,R);

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [32]:
ped = get_pedigree(pedfile);

Finished!


In [33]:
set_random(model,"Animal", ped,G)

In [34]:
set_random(model,"age", 1.0,df=8)

In [22]:
outputMCMCsamples(model,"age")

In [23]:
add_markers(model,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/Github/JWAS.jl/src/5.Datasets/src/../data/testMT/genotype.txt is ,  .


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [24]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0; 0.0]=>0.25,[0.0; 1.0]=>0.25,[0.0; 0.0]=>0.25)
out2 = runMCMC(model,data,methods="BayesC",Pi=Pi,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

Priors for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[17.534247 1.753425
 1.753425 1.753425].


MCMC Information:
methods                                      BayesC
chain length                                   1000
estimatePi                                    false
constraint                                    false
missing_phenotypes                            false
starting value                                false
output sample frequency                          10
printout frequency                              500

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  8.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000



running MCMC for BayesC... 39%|██████████               |  ETA: 0:00:00
Posterior means at iteration: 500
Residual covariance matrix: 
[

In [25]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 12 entries. Keys:
  "Model frequency"
  "Posterior mean of marker effects"
  "MCMC samples for polygenic effects covariance matrix"
  "MCMC samples for: variance of 2:age"
  "Posterior mean of location parameters"
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for: 1:age"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of residual covariance matrix"
  "MCMC samples for: 2:age"
  "MCMC samples for: variance of 1:age"

In [26]:
out2["Posterior mean of marker effects"]

2-element Array{Any,1}:
 5x2 Array{Any,2}:
 "x1"   0.322683 
 "x2"  -0.0713638
 "x3"   0.385389 
 "x4"   0.500715 
 "x5"   0.240846 
 5x2 Array{Any,2}:
 "x1"  -0.0569113
 "x2"   0.211425 
 "x3"  -0.0579064
 "x4"  -0.101527 
 "x5"   0.0516265

In [35]:
JWAS.getMME(model,data)

In [39]:
[JWAS.getNames(model) round(model.mmeLhs,2)]

24x25 Array{Any,2}:
 "1:intercept : intercept"   0.67  …  -0.33  -0.33  -0.33  -0.33   0.0 
 "1:age : 8"                 0.17     -0.33   0.0    0.0    0.0    0.0 
 "1:age : 7"                 0.17      0.0   -0.33   0.0    0.0    0.0 
 "1:age : 3"                 0.17      0.0    0.0   -0.33   0.0    0.0 
 "1:age : 4"                 0.17      0.0    0.0    0.0   -0.33   0.0 
 "1:sex : M"                 0.33  …  -0.33   0.0   -0.33   0.0    0.0 
 "1:sex : F"                 0.33      0.0   -0.33   0.0   -0.33   0.0 
 "1:Animal : S1"             0.17     -0.4   -0.04   0.03   0.03   0.03
 "1:Animal : D1"             0.17     -0.04  -0.4    0.03   0.03   0.03
 "1:Animal : O1"             0.17      0.03   0.03  -0.38   0.0    0.0 
 "1:Animal : O3"             0.17  …   0.03   0.03   0.0   -0.38   0.0 
 "1:Animal : O2"             0.0       0.03   0.03   0.0    0.0   -0.05
 "2:intercept : intercept"  -1.33      1.67   1.67   1.67   1.67   0.0 
 "2:age : 8"                -0.33      1.67 